In [40]:
from bs4 import BeautifulSoup
import requests
import math
from dateutil import parser

In [41]:
def scrapCorpus(source,keyword,nb_docs):
    if source=="emploi-territorial":
        rootLink = "https://www.emploi-territorial.fr"
        # 20 offres par pages
        pages = math.ceil(nb_docs/20)
        url = f"{rootLink}/emploi-mobilite/?adv-search={keyword}&page={pages}"
        response = requests.get(url) 
        soup = BeautifulSoup(response.content, 'html.parser')
        root = soup.find("body")
        offresLinkElems = root.select("div[class*='bloc-lien-offre'] > a[class*='lien-details-offre']")[:nb_docs]    
        links = [rootLink+offresLinkElem.get("href") for offresLinkElem in offresLinkElems]


        corpus = list(dict())

        for link in links:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            root = soup.find("body")

            mainTitle = root.select("h2[class*='set-line-emploi']")[0].text
            company = root.select("div[class*='offre-item-value'] > strong > a")[0].text if root.select("div[class*='offre-item-value'] > strong > a") else "NULL"
            workplace = root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value").text if root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value") else "NULL"
            published_date = root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green").text if root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green") else "NULL"
            contract_type = root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value').text if root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value') else "NULL"
            description = root.select_one('div[class*="offre-item-label"]:contains("Descriptif de l\'emploi") + .set-background-light-gray').text if root.select_one('div[class*="offre-item-label"]:contains("Descriptif de l\'emploi") + .set-background-light-gray') else "NULL"
            long_infos = root.select('div[class*="offre-item-text"]')
            
            description = long_infos[0].text
            missions = long_infos[1].text
            profiles_seeked = long_infos[2].text
            
            corpus.append({"position":mainTitle,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":description,"missions":missions,"profiles_seeked":profiles_seeked})
        return(corpus)


In [42]:
corpus = scrapCorpus(source="emploi-territorial",keyword="data",nb_docs=24)

In [43]:
corpus

[{'position': 'Chef.fe de projets Ingénierie Data',
  'company': 'CONSEIL REGIONAL OCCITANIE ',
  'workplace': 'TOULOUSE',
  'published_date': '28/11/2023',
  'contract_type': "Emploi permanent - vacance d'emploi",
  'description': "\nLa Direction de l'Information et du Numérique (DIN) est actrice de l'innovation, de la donnée et de la transformation numérique de la collectivité. \n\nLe Service Données et Territoires a pour mission d'animer et de mettre en œuvre la politique de valorisation de la donnée de la collectivité. Son périmètre englobe le système d'information géographique, l'informatique décisionnelle, les enjeux d'innovation liés aux données massives, le développement de l'IA et de l'Open data. \n\nLe.la chef.fe de projet Ingénierie Data adresse les enjeux de l'accessibilité de la donnée au sein de la collectivité, entre partenaires, entre applicatifs et envers le citoyen. Sa mission se décline en plusieurs niveaux : \n*\tL'Open Data ; \n*\tLes échanges partenariaux ;\n*\tLe

In [44]:
print(len(corpus))

24
